In [1]:
# Dependencies and Setup
import requests
import time
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import sklearn.datasets as dta
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress
import seaborn as sns

In [2]:
# Set the base URL
covid_url = 'https://api.opencovid.ca/timeseries?geo=pt&fill=false&version=true&pt_names=canonical&hr_names=hruid&fmt=json'
covid_confirmed_cases_response = requests.get(covid_url).json()
# Define an empty list to fetch the covid confirmed cases
covid_cases_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_confirmed_cases_response['data']['cases']:
    covid_cases_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative confirmed cases": case['value'], "Confirmed cases per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [3]:
covid_confirmed_cases_data_df = pd.DataFrame(covid_cases_data)
# Show Record Count
covid_confirmed_cases_data_df

,Province,Date,Cumulative confirmed cases,Confirmed cases per day
0,Alberta,2020-03-05,1,1
1,Alberta,2020-03-06,1,0
2,Alberta,2020-03-07,2,1
3,Alberta,2020-03-08,3,1
4,Alberta,2020-03-09,7,4
...,...,...,...,...
14390,Yukon,2022-11-06,5578,5
14391,Yukon,2022-11-07,5581,3
14392,Yukon,2022-11-08,5582,1
14393,Yukon,2022-11-09,5582,0


In [4]:
# Set the API base URL
covid_url = 'https://api.opencovid.ca/timeseries?geo=pt&fill=false&version=true&pt_names=canonical&hr_names=hruid&fmt=json'
covid_deaths_response = requests.get(covid_url).json()
# Define an empty list to fetch the death data
covid_death_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_deaths_response['data']['deaths']:
    covid_death_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative death cases": case['value'], "Deaths per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [5]:
death_data_df = pd.DataFrame(covid_death_data)
death_data_df

,Province,Date,Cumulative death cases,Deaths per day
0,Alberta,2020-03-08,0,0
1,Alberta,2020-03-09,0,0
2,Alberta,2020-03-10,0,0
3,Alberta,2020-03-11,0,0
4,Alberta,2020-03-12,0,0
...,...,...,...,...
14292,Yukon,2022-11-01,32,0
14293,Yukon,2022-11-02,32,0
14294,Yukon,2022-11-03,32,0
14295,Yukon,2022-11-04,32,0


In [6]:
# Set the API base URL
covid_url = 'https://api.opencovid.ca/timeseries?geo=pt&fill=false&version=true&pt_names=canonical&hr_names=hruid&fmt=json'
covid_hospitalization_response = requests.get(covid_url).json()
# Define an empty list to fetch the death data
hospitalization_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_hospitalization_response['data']['hospitalizations']:
    hospitalization_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative hospitalizations cases": case['value'], "hospitalizations per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [7]:
hospitalization_data_df = pd.DataFrame(hospitalization_data)
hospitalization_data_df

,Province,Date,Cumulative hospitalizations cases,hospitalizations per day
0,Alberta,2020-01-25,0,0
1,Alberta,2020-01-26,0,0
2,Alberta,2020-01-27,0,0
3,Alberta,2020-01-28,0,0
4,Alberta,2020-01-29,0,0
...,...,...,...,...
14000,Yukon,2022-11-13,0,0
14001,Yukon,2022-11-14,0,0
14002,Yukon,2022-11-15,0,0
14003,Yukon,2022-11-16,0,0


In [8]:
# Set the base URL
covid_url = 'https://api.opencovid.ca/timeseries?geo=pt&fill=false&version=true&pt_names=canonical&hr_names=hruid&fmt=json'
covid_vaccinationdose1_coverage_response = requests.get(covid_url).json()
# Define an empty list to fetch the covid confirmed cases
covid_vaccinationdose1_coverage_data = []
# Print to logger
print("Beginning Data Retrieval     ")
for case in covid_vaccinationdose1_coverage_response['data']['vaccine_coverage_dose_1']:
    covid_vaccinationdose1_coverage_data.append({"Province": case['region'], "Date" : case['date'], "Cumulative vaccine coverage dose1": case['value'], "Vaccine coverage dose1 per day": case['value_daily']})            
# Indicate that Data Loading is complete 
print("Data Retrieval Complete      ")

Beginning Data Retrieval     
Data Retrieval Complete      


In [9]:
covid_vaccinationdose1_coverage_data_df = pd.DataFrame(covid_vaccinationdose1_coverage_data)
# Show Record Count
covid_vaccinationdose1_coverage_data_df.head(3)

,Province,Date,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day
0,Alberta,2020-12-19,0.0,0.0
1,Alberta,2020-12-20,0.0,0.0
2,Alberta,2020-12-21,0.0,0.0


In [10]:
# Mering the three DataFrames named as covid_confirmed_cases_data_df, death_data_df and hospitalization_data_df on columns Province and Date as they are common columns in all of the dataframes
# covid_combined_df = pd.merge(covid_confirmed_cases_data_df, pd.merge(death_data_df, hospitalization_data_df, on=['Province', 'Date']), on=['Province', 'Date'])
# covid_combined_df
covid_combined_df = covid_confirmed_cases_data_df.merge(death_data_df, on=['Province', 'Date']).merge(hospitalization_data_df, on=['Province', 'Date']).merge(covid_vaccinationdose1_coverage_data_df, on=['Province', 'Date'])
covid_combined_df.describe()

,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day
count,9.266000e+03,9266.000000,9266.000000,9266.000000,9266.000000,9266.000000,9266.000000,9266.000000
mean,2.206529e+05,422.826139,2750.898878,3.817721,306.923160,-0.005936,65.462174,0.119264
std,3.549360e+05,1155.429091,4287.182875,9.558755,504.427551,22.991297,28.684811,0.684302
min,2.400000e+01,-4.000000,0.000000,-12.000000,0.000000,-314.000000,0.000000,-9.100000
25%,2.435250e+03,0.000000,21.000000,0.000000,0.000000,0.000000,60.275000,0.000000
50%,5.581200e+04,58.000000,571.500000,0.000000,69.000000,0.000000,79.000000,0.000000
75%,3.092628e+05,409.000000,3300.000000,3.000000,362.750000,0.000000,83.900000,0.000000
max,1.614845e+06,19509.000000,16485.000000,112.000000,4190.000000,272.000000,95.700000,17.500000


In [11]:
covid_combined_final_df = covid_combined_df[(covid_combined_df['Cumulative confirmed cases'] >= 0) & (covid_combined_df['Confirmed cases per day'] >= 0) & (covid_combined_df['Cumulative death cases'] >= 0) & (covid_combined_df['Deaths per day'] >= 0) & (covid_combined_df['Cumulative hospitalizations cases'] >= 0)& (covid_combined_df['hospitalizations per day'] >= 0)& (covid_combined_df['Cumulative vaccine coverage dose1'] >= 0) & (covid_combined_df['Vaccine coverage dose1 per day'] >= 0)]
covid_combined_final_df = covid_combined_final_df.dropna()
covid_combined_final_df.describe()

,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day
count,7.358000e+03,7358.000000,7358.000000,7358.000000,7358.00000,7358.000000,7358.000000,7358.000000
mean,1.726651e+05,358.907855,2037.549606,2.787170,241.35458,4.938298,66.931530,0.115072
std,3.013404e+05,1188.076589,3579.853194,8.635528,432.91207,18.411284,27.925885,0.648600
min,2.400000e+01,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,1.649000e+03,0.000000,12.000000,0.000000,0.00000,0.000000,63.300000,0.000000
50%,4.712300e+04,16.000000,258.000000,0.000000,33.00000,0.000000,79.500000,0.000000
75%,1.551062e+05,232.750000,2280.250000,1.000000,292.00000,0.000000,84.000000,0.000000
max,1.614711e+06,19509.000000,16483.000000,112.000000,4190.00000,272.000000,95.700000,17.500000


In [12]:
covid_combined_final_df['Mortality rate'] = round(((covid_combined_final_df['Cumulative death cases'] / covid_combined_final_df['Cumulative confirmed cases']) * 100), 3)
covid_combined_final_df['Daily mortality rate'] = round(((covid_combined_final_df['Deaths per day'] / covid_combined_final_df['Confirmed cases per day']) * 100), 3)
covid_combined_final_df.describe()

,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day,Mortality rate,Daily mortality rate
count,7.358000e+03,7358.000000,7358.000000,7358.000000,7358.00000,7358.000000,7358.000000,7358.000000,7358.000000,4882.00
mean,1.726651e+05,358.907855,2037.549606,2.787170,241.35458,4.938298,66.931530,0.115072,0.981959,inf
std,3.013404e+05,1188.076589,3579.853194,8.635528,432.91207,18.411284,27.925885,0.648600,0.755054,NaN
min,2.400000e+01,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00
25%,1.649000e+03,0.000000,12.000000,0.000000,0.00000,0.000000,63.300000,0.000000,0.512000,0.00
50%,4.712300e+04,16.000000,258.000000,0.000000,33.00000,0.000000,79.500000,0.000000,0.894500,0.00
75%,1.551062e+05,232.750000,2280.250000,1.000000,292.00000,0.000000,84.000000,0.000000,1.312000,0.73
max,1.614711e+06,19509.000000,16483.000000,112.000000,4190.00000,272.000000,95.700000,17.500000,4.505000,inf


In [13]:
# sort the DataFrame by date in ascending order as covid_combined_final_sorted
covid_combined_final_df['Date'] = pd.to_datetime(covid_combined_final_df['Date'])
covid_combined_final_sorted = covid_combined_final_df.sort_values(by='Date')

In [14]:
# fill missing values with zero
covid_combined_final_sorted_filled = covid_combined_final_sorted.fillna(value=0)
# remove inf and replace with zero in column 'A'
covid_combined_final_sorted_filled['Daily mortality rate'] = covid_combined_final_sorted_filled['Daily mortality rate'].replace([np.inf, -np.inf], 0)

In [15]:
covid_combined_final_sorted_filled.head()

,Province,Date,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day,Mortality rate,Daily mortality rate
0,Alberta,2020-12-19,89993,1306,851,10,777,18,0.0,0.0,0.946,0.766
3981,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.000,0.000
2560,New Brunswick,2020-12-19,574,1,8,0,3,0,0.0,0.0,1.394,0.000
7880,Saskatchewan,2020-12-19,13329,252,115,8,127,6,0.1,0.1,0.863,3.175
8579,Yukon,2020-12-19,69,0,1,0,0,0,0.0,0.0,1.449,0.000


In [16]:
covid_combined_final_sorted_filled.to_csv("Covid19_data_project.csv", index=False)

In [17]:
covid_dataframe_df = covid_combined_final_sorted_filled
covid_dataframe_df.head(5)

,Province,Date,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day,Mortality rate,Daily mortality rate
0,Alberta,2020-12-19,89993,1306,851,10,777,18,0.0,0.0,0.946,0.766
3981,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.000,0.000
2560,New Brunswick,2020-12-19,574,1,8,0,3,0,0.0,0.0,1.394,0.000
7880,Saskatchewan,2020-12-19,13329,252,115,8,127,6,0.1,0.1,0.863,3.175
8579,Yukon,2020-12-19,69,0,1,0,0,0,0.0,0.0,1.449,0.000


In [18]:
# Assumptions made recovered data/Cured data was not available in the API so we assumed to use the cumulative hospitalizations cases to calculate the cumulative active cases
covid_dataframe_df['Cumulative Active Cases'] = covid_dataframe_df['Cumulative confirmed cases'] - (covid_dataframe_df['Cumulative death cases'] + covid_dataframe_df['Cumulative hospitalizations cases'])
covid_dataframe_df['Active Cases per day'] = covid_dataframe_df['Confirmed cases per day'] - (covid_dataframe_df['Deaths per day'] + covid_dataframe_df['hospitalizations per day'])
covid_dataframe_df.head(3)

,Province,Date,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day,Mortality rate,Daily mortality rate,Cumulative Active Cases,Active Cases per day
0,Alberta,2020-12-19,89993,1306,851,10,777,18,0.0,0.0,0.946,0.766,88365,1278
3981,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.000,0.000,24,0
2560,New Brunswick,2020-12-19,574,1,8,0,3,0,0.0,0.0,1.394,0.000,563,1


In [33]:
Active_cases_per_province = covid_dataframe_df.groupby('Province').max()[['Cumulative Active Cases', 'Date']].sort_values(['Cumulative Active Cases'], ascending = False).reset_index()
# Active_cases_per_province = Active_cases_per_province.iloc[0:5]
Active_cases_per_province_df = Active_cases_per_province[['Province', 'Cumulative Active Cases']]
Active_cases_per_province_df = Active_cases_per_province_df.set_index('Province')
Active_cases_per_province_df.head(13)

,Cumulative Active Cases
Province,
Ontario,1597547
Quebec,1161158
Alberta,625662
British Columbia,393391
Manitoba,153062
Saskatchewan,145617
New Brunswick,83576
Nova Scotia,73053
Prince Edward Island,53893


In [20]:
Vaccine_dose1_df = pd.read_csv('C:/Users/panka/OneDrive/Desktop/Class Folder/Github/COVID-19_project_work/vaccination-coverage-byAgeAndSex-overTimeDownload.csv', delimiter=',', header=0, na_values='NaN')
Vaccine_dose1_df['Date'] = pd.to_datetime(Vaccine_dose1_df['Date'])
Vaccine_dose1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81228 entries, 0 to 81227
Data columns (total 5 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Province                                     81228 non-null  object        
 1   Date                                         81228 non-null  datetime64[ns]
 2   Sex                                          81228 non-null  object        
 3   Age                                          81228 non-null  object        
 4   Cummulative number of people (Vaccinedose1)  81228 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


In [29]:
merged_df = pd.merge(covid_dataframe_df, Vaccine_dose1_df, on=['Province', 'Date'], how="inner")
# Remove NaN values from specific columns
# Assuming you want to remove NaN values and other common missing value representations from the 'column_name' column
missing_values = ['na', 'N/A', 'null', '']

# Replace missing value representations with NaN
merged_df['Cummulative number of people (Vaccinedose1)'] = merged_df['Cummulative number of people (Vaccinedose1)'].replace(missing_values, float('nan'))

# Drop NaN values from the column
merged_df.dropna(subset=['Cummulative number of people (Vaccinedose1)'], inplace=True)

merged_df.to_csv("Covid19_data_vaccine.csv", index=False)
# merged_df.sample(100)

In [30]:
Active_cases_per_province = merged_df.groupby('Province').max()[['Cumulative Active Cases', 'Date']].sort_values(['Cumulative Active Cases'], ascending = False).reset_index()
# Active_cases_per_province = Active_cases_per_province.iloc[0:5]
Active_cases_per_province_df = Active_cases_per_province[['Province', 'Cumulative Active Cases']]
Active_cases_per_province_df = Active_cases_per_province_df.set_index('Province')
Active_cases_per_province_df.head(13)

,Cumulative Active Cases
Province,
Ontario,1172465
Quebec,1161158
Alberta,625662
British Columbia,392138
Manitoba,153062
Saskatchewan,145182
New Brunswick,82764
Nova Scotia,72067
Prince Edward Island,53715


In [26]:
# Assuming you want to convert the 'Cummulative number of people (Vaccinedose1)' column to int64
merged_df['Cummulative number of people (Vaccinedose1)'] = merged_df['Cummulative number of people (Vaccinedose1)'].astype('int64')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53792 entries, 0 to 54145
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Province                                     53792 non-null  object        
 1   Date                                         53792 non-null  datetime64[ns]
 2   Cumulative confirmed cases                   53792 non-null  int64         
 3   Confirmed cases per day                      53792 non-null  int64         
 4   Cumulative death cases                       53792 non-null  int64         
 5   Deaths per day                               53792 non-null  int64         
 6   Cumulative hospitalizations cases            53792 non-null  int64         
 7   hospitalizations per day                     53792 non-null  int64         
 8   Cumulative vaccine coverage dose1            53792 non-null  float64       


In [35]:
merged_df.head(5)

,Province,Date,Cumulative confirmed cases,Confirmed cases per day,Cumulative death cases,Deaths per day,Cumulative hospitalizations cases,hospitalizations per day,Cumulative vaccine coverage dose1,Vaccine coverage dose1 per day,Mortality rate,Daily mortality rate,Cumulative Active Cases,Active Cases per day,Sex,Age,Cummulative number of people (Vaccinedose1)
0,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.0,0.0,24,0,All sexes,0–15,0
1,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.0,0.0,24,0,All sexes,16–69,0
2,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.0,0.0,24,0,All sexes,70–74,0
3,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.0,0.0,24,0,All sexes,75–79,0
4,Northwest Territories,2020-12-19,24,0,0,0,0,0,0.0,0.0,0.0,0.0,24,0,All sexes,80+,0


In [44]:
import matplotlib.pyplot as plt

# Group the data by 'Age' and calculate the sum of 'Cummulative number of people (Vaccinedose1)'
grouped_df = merged_df.groupby('Age')['Cummulative number of people (Vaccinedose1)'].sum().reset_index()

# Sort the data by 'Age' column
grouped_df = grouped_df.sort_values('Age')

# Plot the data
plt.figure(figsize=(12, 8))
plt.bar(grouped_df['Age'], grouped_df['Cummulative number of people (Vaccinedose1)'])
plt.xlabel('Age')
plt.ylabel('Cumulative Number of People Vaccinated (Vaccinedose1)')
plt.title('Cumulative Number of People Vaccinated by Age Group')
plt.xticks(rotation=90)

# Save the plot as a PNG image
plt.savefig('vaccination_plot.png', bbox_inches='tight')
plt.close()

ValueError: Image size of 151153x774 pixels is too large. It must be less than 2^16 in each direction.

Error in callback <function _draw_all_if_interactive at 0x000001E99A02F3A0> (for post_execute):


RuntimeError: In draw_glyphs_to_bitmap: Could not convert glyph to bitmap (raster overflow; error code 0x62)

ValueError: Image size of 151153x774 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1200x800 with 1 Axes>